## Interpreter

**Typ**: behawioralny \
**Zakres**: klasowy

<div style="border: solid 1px;padding: 20px;text-align: center">
    Wzorzec <b>interpreter</b> definiuje reprezentację gramatyki danego języka wraz z interpreterem, który używa tej reprezentacji do interpretowania zdań w tym języku.
</div>

### Problem - interpretowanie wyrażeń

Chcemy ewaluować wyrażenia matematyczne:
- `"3 + 5"` → `8`
- `"10 - 4"` → `6`
- `"3 + 5 - 2"` → `6`
- `"x + y"` gdzie `x=10, y=20` → `30`

**Problem:**
- Jak reprezentować wyrażenia?
- Jak ewaluować złożone wyrażenia?
- Jak obsługiwać zmienne?

### Naiwne podejście - jeden wielki parser

In [ ]:
def evaluate(expression):
    """❌ Naiwna funkcja parsująca"""
    
    # Usuń spacje
    expression = expression.replace(" ", "")
    
    # ❌ Ogromny if/elif dla każdego przypadku
    if "+" in expression:
        parts = expression.split("+")
        return int(parts[0]) + int(parts[1])
    elif "-" in expression:
        parts = expression.split("-")
        return int(parts[0]) - int(parts[1])
    else:
        return int(expression)

In [ ]:
print(evaluate("3 + 5"))   # 8
print(evaluate("10 - 4"))  # 6
# print(evaluate("3 + 5 - 2"))  # ❌ Nie zadziała!
# print(evaluate("x + y"))       # ❌ Nie zadziała!

**Problemy:**
- ❌ **Nie skaluje się** - złożone wyrażenia wymagają coraz bardziej skomplikowanej logiki
- ❌ **Brak struktury** - trudno reprezentować drzewo wyrażeń
- ❌ **Trudność rozszerzania** - dodanie mnożenia/dzielenia = przepisanie kodu
- ❌ **Brak wsparcia dla zmiennych**

### Rozwiązanie - wzorzec Interpreter

**Idea:** Reprezentuj wyrażenie jako **drzewo obiektów**. Każdy element gramatyki = osobna klasa.

```
"3 + 5" → AddExpression(Number(3), Number(5))

     +
    / \
   3   5
```

### Krok 1: Abstract Expression - interfejs

In [ ]:
from abc import ABC, abstractmethod

class Expression(ABC):
    """Abstrakcyjne wyrażenie - interfejs dla wszystkich wyrażeń"""
    
    @abstractmethod
    def interpret(self, context):
        """Ewaluuj wyrażenie w danym kontekście"""
        pass

### Krok 2: Terminal Expressions - liście drzewa

In [ ]:
class Number(Expression):
    """Terminal Expression - liczba (liść drzewa)"""
    
    def __init__(self, value):
        self.value = value
    
    def interpret(self, context):
        """Zwróć wartość"""
        return self.value


class Variable(Expression):
    """Terminal Expression - zmienna (liść drzewa)"""
    
    def __init__(self, name):
        self.name = name
    
    def interpret(self, context):
        """Pobierz wartość z kontekstu"""
        return context.get(self.name, 0)

**Terminal Expressions:**
- Liście drzewa wyrażeń
- Nie zawierają innych wyrażeń
- Bezpośrednio zwracają wartość

### Krok 3: Nonterminal Expressions - operatory

In [ ]:
class Add(Expression):
    """Nonterminal Expression - dodawanie"""
    
    def __init__(self, left: Expression, right: Expression):
        self.left = left
        self.right = right
    
    def interpret(self, context):
        """Ewaluuj lewą i prawą stronę, dodaj"""
        return self.left.interpret(context) + self.right.interpret(context)


class Subtract(Expression):
    """Nonterminal Expression - odejmowanie"""
    
    def __init__(self, left: Expression, right: Expression):
        self.left = left
        self.right = right
    
    def interpret(self, context):
        """Ewaluuj lewą i prawą stronę, odejmij"""
        return self.left.interpret(context) - self.right.interpret(context)


class Multiply(Expression):
    """Nonterminal Expression - mnożenie"""
    
    def __init__(self, left: Expression, right: Expression):
        self.left = left
        self.right = right
    
    def interpret(self, context):
        return self.left.interpret(context) * self.right.interpret(context)

**Nonterminal Expressions:**
- Węzły wewnętrzne drzewa
- Zawierają inne wyrażenia (`left`, `right`)
- Rekursywnie ewaluują podwyrażenia

### Krok 4: Użycie - budowanie drzewa wyrażeń

In [ ]:
# Wyrażenie: 3 + 5
#      +
#     / \
#    3   5
expr1 = Add(Number(3), Number(5))
print(f"3 + 5 = {expr1.interpret({})}")

# Wyrażenie: 10 - 4
expr2 = Subtract(Number(10), Number(4))
print(f"10 - 4 = {expr2.interpret({})}")

# Wyrażenie: (3 + 5) - 2
#        -
#       / \
#      +   2
#     / \
#    3   5
expr3 = Subtract(
    Add(Number(3), Number(5)),
    Number(2)
)
print(f"(3 + 5) - 2 = {expr3.interpret({})}")

### Krok 5: Context - zmienne

In [ ]:
# Wyrażenie: x + y
#      +
#     / \
#    x   y
expr4 = Add(Variable("x"), Variable("y"))

# Kontekst - wartości zmiennych
context = {"x": 10, "y": 20}

print(f"x + y = {expr4.interpret(context)} (gdzie x={context['x']}, y={context['y']})")

# Zmiana kontekstu
context2 = {"x": 100, "y": 200}
print(f"x + y = {expr4.interpret(context2)} (gdzie x={context2['x']}, y={context2['y']})")

### Krok 6: Złożone wyrażenie

In [ ]:
# Wyrażenie: (x + 5) * (y - 2)
#           *
#         /   \
#        +     -
#       / \   / \
#      x   5 y   2

expr5 = Multiply(
    Add(Variable("x"), Number(5)),
    Subtract(Variable("y"), Number(2))
)

context = {"x": 3, "y": 10}
print(f"(x + 5) * (y - 2) = {expr5.interpret(context)}")
print(f"gdzie x={context['x']}, y={context['y']}")
print(f"Obliczenie: (3 + 5) * (10 - 2) = 8 * 8 = 64")

**Zalety:**
- ✅ **Struktura** - wyrażenie reprezentowane jako drzewo
- ✅ **Rozszerzalność** - nowy operator = nowa klasa
- ✅ **Rekursja** - złożone wyrażenia ewaluowane automatycznie
- ✅ **Separacja** - gramatyka oddzielona od interpretacji

## Jak to działa? - rekursyjna ewaluacja

```python
# Wyrażenie: (3 + 5) - 2
expr = Subtract(Add(Number(3), Number(5)), Number(2))

expr.interpret({})
    ↓
    # Subtract.interpret()
    return self.left.interpret({}) - self.right.interpret({})
           ↓                          ↓
           Add.interpret({})          Number(2).interpret({})
           ↓                          ↓
           Number(3).interpret({}) + Number(5).interpret({})     return 2
           ↓                          ↓
           return 3                   return 5
           ↓
           return 3 + 5 = 8
    ↓
    return 8 - 2 = 6
```

**Kluczowy mechanizm:**
- Każde wyrażenie wie jak się ewaluować
- Nonterminal expressions rekursywnie wywołują `interpret()` na podwyrażeniach
- Terminal expressions zwracają wartość bezpośrednio

## Struktura wzorca

**Elementy wzorca Interpreter:**

1. **AbstractExpression** - `Expression`
   - Interfejs dla wszystkich wyrażeń
   - Metoda `interpret(context)`

2. **TerminalExpression** - `Number`, `Variable`
   - Liście drzewa wyrażeń
   - Bezpośrednio zwracają wartość

3. **NonterminalExpression** - `Add`, `Subtract`, `Multiply`
   - Węzły wewnętrzne drzewa
   - Zawierają inne wyrażenia
   - Rekursywnie ewaluują podwyrażenia

4. **Context** - słownik `{"x": 10, "y": 20}`
   - Przechowuje wartości zmiennych
   - Przekazywany podczas interpretacji

**Kluczowa właściwość:**
> Każdy element gramatyki = osobna klasa implementująca `interpret()`

## Przykład 2 - Wyrażenia logiczne (Boolean)

In [ ]:
from abc import ABC, abstractmethod

# ════════════════════════════════════════════════════════════
# Abstract Expression
# ════════════════════════════════════════════════════════════
class BooleanExpression(ABC):
    @abstractmethod
    def interpret(self, context):
        pass


# ════════════════════════════════════════════════════════════
# Terminal Expressions
# ════════════════════════════════════════════════════════════
class Constant(BooleanExpression):
    """Terminal - True/False"""
    
    def __init__(self, value: bool):
        self.value = value
    
    def interpret(self, context):
        return self.value


class BooleanVariable(BooleanExpression):
    """Terminal - zmienna boolean"""
    
    def __init__(self, name: str):
        self.name = name
    
    def interpret(self, context):
        return context.get(self.name, False)


# ════════════════════════════════════════════════════════════
# Nonterminal Expressions
# ════════════════════════════════════════════════════════════
class And(BooleanExpression):
    """Nonterminal - AND"""
    
    def __init__(self, left: BooleanExpression, right: BooleanExpression):
        self.left = left
        self.right = right
    
    def interpret(self, context):
        return self.left.interpret(context) and self.right.interpret(context)


class Or(BooleanExpression):
    """Nonterminal - OR"""
    
    def __init__(self, left: BooleanExpression, right: BooleanExpression):
        self.left = left
        self.right = right
    
    def interpret(self, context):
        return self.left.interpret(context) or self.right.interpret(context)


class Not(BooleanExpression):
    """Nonterminal - NOT"""
    
    def __init__(self, expression: BooleanExpression):
        self.expression = expression
    
    def interpret(self, context):
        return not self.expression.interpret(context)

In [ ]:
# Wyrażenie: (isAdmin AND isActive) OR isSuperUser
#              OR
#            /    \
#          AND    isSuperUser
#         /   \
#   isAdmin   isActive

expr = Or(
    And(
        BooleanVariable("isAdmin"),
        BooleanVariable("isActive")
    ),
    BooleanVariable("isSuperUser")
)

# Test 1
context1 = {"isAdmin": True, "isActive": True, "isSuperUser": False}
print(f"Kontekst: {context1}")
print(f"Wynik: {expr.interpret(context1)}\n")

# Test 2
context2 = {"isAdmin": True, "isActive": False, "isSuperUser": False}
print(f"Kontekst: {context2}")
print(f"Wynik: {expr.interpret(context2)}\n")

# Test 3
context3 = {"isAdmin": False, "isActive": False, "isSuperUser": True}
print(f"Kontekst: {context3}")
print(f"Wynik: {expr.interpret(context3)}")

**Zastosowanie:**
- Systemy reguł biznesowych
- Walidacja uprawnień
- Filtry i zapytania

## Interpreter + Parser - parsowanie stringów

W praktyce często potrzebujemy **parsera** do konwersji stringa na drzewo wyrażeń:

In [ ]:
def parse_simple(expression_str):
    """Prosty parser dla wyrażeń typu '3 + 5' lub 'x + y'"""
    
    expression_str = expression_str.strip()
    
    # Dodawanie
    if "+" in expression_str:
        parts = expression_str.split("+")
        left = parse_simple(parts[0].strip())
        right = parse_simple(parts[1].strip())
        return Add(left, right)
    
    # Odejmowanie
    elif "-" in expression_str:
        parts = expression_str.split("-")
        left = parse_simple(parts[0].strip())
        right = parse_simple(parts[1].strip())
        return Subtract(left, right)
    
    # Liczba lub zmienna
    else:
        if expression_str.isdigit():
            return Number(int(expression_str))
        else:
            return Variable(expression_str)

In [ ]:
# Parsowanie i ewaluacja
expr_tree = parse_simple("x + 10")
context = {"x": 5}

print(f"Wyrażenie: 'x + 10'")
print(f"Kontekst: {context}")
print(f"Wynik: {expr_tree.interpret(context)}")

**Uwaga:**
- Powyższy parser jest **bardzo uproszczony**
- W praktyce używa się bibliotek do parsowania (np. `pyparsing`, `lark`, `PLY`)
- Wzorzec Interpreter koncentruje się na **reprezentacji i ewaluacji**, nie parsowaniu

## Interpreter vs Composite

| Aspekt | Interpreter | Composite |
|--------|-------------|----------|
| **Cel** | **Interpretacja** gramatyki/języka | Reprezentacja hierarchii **część-całość** |
| **Operacja** | `interpret()` - ewaluacja | `operation()` - dowolna operacja |
| **Typ wzorca** | Behawioralny | Strukturalny |
| **Context** | Używa kontekstu dla zmiennych | Nie używa kontekstu |
| **Przykład** | Wyrażenia matematyczne, DSL | System plików, GUI komponenty |

**Podobieństwa:**
- Oba używają struktury drzewa
- Oba mają liście (terminal) i węzły (nonterminal)
- Oba używają rekursji

**Różnica:**
```python
# Interpreter - INTERPRETACJA wyrażenia
expr = Add(Number(3), Number(5))
result = expr.interpret({})  # Ewaluuje do 8

# Composite - OPERACJA na hierarchii
folder = Folder("root")
folder.add(File("a.txt"))
folder.get_size()  # Operacja, nie interpretacja
```

## Kiedy używać wzorca Interpreter?

Wzorzec Interpreter stosuj gdy:

1. **Masz prosty język/gramatykę do zinterpretowania**
   - Wyrażenia matematyczne, logiczne
   - Proste języki zapytań

2. **Gramatyka jest stabilna**
   - Zmiany w gramatyce wymagają modyfikacji klas

3. **Wydajność nie jest krytyczna**
   - Interpreter jest wolniejszy niż skompilowany kod

4. **Chcesz reprezentować gramatykę jako klasy**
   - Łatwe rozszerzanie o nowe operacje

**Przykłady praktyczne:**
- Wyrażenia matematyczne
- Wyrażenia regularne (proste)
- Języki zapytań (SQL-like dla małych danych)
- Języki konfiguracyjne
- Reguły biznesowe
- DSL (Domain-Specific Languages)

## Wady wzorca Interpreter

⚠️ **Trudność dla złożonych gramatyk:**
- Każdy element gramatyki = osobna klasa
- Złożona gramatyka = dziesiątki klas

⚠️ **Niska wydajność:**
- Rekursywne wywołania metod
- Wolniejsze niż skompilowany kod

⚠️ **Trudność utrzymania:**
- Zmiana gramatyki wymaga modyfikacji wielu klas

**Alternatywy:**
- Dla złożonych języków: generatory parserów (ANTLR, Yacc)
- Dla wydajności: kompilacja do bytecode

## Przykład 3 - Język zapytań (Query DSL)

In [ ]:
from abc import ABC, abstractmethod

# Abstract Expression
class QueryExpression(ABC):
    @abstractmethod
    def interpret(self, data):
        """Filtruj dane według wyrażenia"""
        pass


# Terminal Expressions
class Equals(QueryExpression):
    """Terminal - równość (pole == wartość)"""
    
    def __init__(self, field, value):
        self.field = field
        self.value = value
    
    def interpret(self, data):
        return [item for item in data if item.get(self.field) == self.value]


class GreaterThan(QueryExpression):
    """Terminal - większe niż (pole > wartość)"""
    
    def __init__(self, field, value):
        self.field = field
        self.value = value
    
    def interpret(self, data):
        return [item for item in data if item.get(self.field, 0) > self.value]


# Nonterminal Expressions
class AndQuery(QueryExpression):
    """Nonterminal - AND"""
    
    def __init__(self, left: QueryExpression, right: QueryExpression):
        self.left = left
        self.right = right
    
    def interpret(self, data):
        # Filtruj lewym, potem prawym
        result = self.left.interpret(data)
        return self.right.interpret(result)


class OrQuery(QueryExpression):
    """Nonterminal - OR"""
    
    def __init__(self, left: QueryExpression, right: QueryExpression):
        self.left = left
        self.right = right
    
    def interpret(self, data):
        # Połącz wyniki, usuń duplikaty
        left_result = self.left.interpret(data)
        right_result = self.right.interpret(data)
        
        # Użyj id() do identyfikacji unikalnych obiektów
        seen = set(id(item) for item in left_result)
        result = left_result[:]
        
        for item in right_result:
            if id(item) not in seen:
                result.append(item)
        
        return result

In [ ]:
# Dane - lista użytkowników
users = [
    {"name": "Alice", "age": 25, "role": "admin"},
    {"name": "Bob", "age": 30, "role": "user"},
    {"name": "Charlie", "age": 35, "role": "admin"},
    {"name": "Diana", "age": 28, "role": "user"},
]

# Zapytanie: age > 25 AND role == "admin"
query1 = AndQuery(
    GreaterThan("age", 25),
    Equals("role", "admin")
)

print("Zapytanie: age > 25 AND role == 'admin'")
result = query1.interpret(users)
for user in result:
    print(f"  {user}")

print()

# Zapytanie: age > 30 OR role == "admin"
query2 = OrQuery(
    GreaterThan("age", 30),
    Equals("role", "admin")
)

print("Zapytanie: age > 30 OR role == 'admin'")
result = query2.interpret(users)
for user in result:
    print(f"  {user}")

## Podsumowanie

Wzorzec Interpreter:
- ✅ **Struktura** - wyrażenia jako drzewo klas
- ✅ **Rozszerzalność** - nowa operacja = nowa klasa
- ✅ **Separacja** - gramatyka oddzielona od ewaluacji
- ✅ **Rekursja** - złożone wyrażenia ewaluowane automatycznie
- ⚠️ **Złożoność** - wiele klas dla skomplikowanej gramatyki
- ⚠️ **Wydajność** - wolniejsze niż skompilowany kod
- ⚠️ **Utrzymanie** - zmiana gramatyki wymaga modyfikacji klas

**Kluczowa idea:**
> **Każdy element gramatyki = osobna klasa** z metodą `interpret()`

**Struktura:**
```python
# Abstract Expression
class Expression(ABC):
    @abstractmethod
    def interpret(self, context):
        pass

# Terminal Expression (liść)
class Number(Expression):
    def __init__(self, value):
        self.value = value
    
    def interpret(self, context):
        return self.value  # Bezpośrednio zwraca wartość

# Nonterminal Expression (węzeł)
class Add(Expression):
    def __init__(self, left: Expression, right: Expression):
        self.left = left
        self.right = right
    
    def interpret(self, context):
        # Rekursywnie ewaluuje podwyrażenia
        return self.left.interpret(context) + self.right.interpret(context)

# Użycie
expr = Add(Number(3), Number(5))  # Drzewo: 3 + 5
result = expr.interpret({})        # Ewaluacja: 8
```

**Istota wzorca:**
- **Drzewo wyrażeń:** Terminal (liście) + Nonterminal (węzły)
- **Rekursyjna ewaluacja:** Każdy węzeł wywołuje `interpret()` na dzieciach
- **Context:** Przechowuje wartości zmiennych
- **Separacja:** Gramatyka (klasy) vs Parsowanie (oddzielna logika)

**Kiedy stosować:**
- ✅ Prosty język/gramatyka
- ✅ Stabilna gramatyka
- ✅ Wydajność nie jest krytyczna
- ❌ Złożona gramatyka (użyj generatora parserów)
- ❌ Wymagana wysoka wydajność (użyj kompilacji)

**Przykłady zastosowań:**
- Wyrażenia matematyczne
- Wyrażenia logiczne (boolean)
- Języki zapytań (query DSL)
- Proste języki konfiguracyjne
- Reguły biznesowe